In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.7MB/s 


In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [4]:
df = pd.read_csv('data/men_shoes.csv', low_memory = False)

In [5]:
def run_model(feats, model = DecisionTreeRegressor(max_depth = 5) ):
    X = df[ feats ].values
    y = df[ 'prices_amountmin' ].values
    scores = cross_val_score(model, X, y, scoring = 'neg_mean_absolute_error')

    return  np.mean( scores ), np.std( scores )

In [6]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower() ).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [7]:
model = RandomForestRegressor(max_depth = 5, n_estimators = 100, random_state = 0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [8]:
df.features.values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       ...,
       '[{"key":"Gender","value":["Unisex"]},{"key":"Frame Style","value":["Wrap"]},{"key":"Polarized Lenses","value":["Polarized lenses"]},{"key":"Color","value":["Multi-color"]}]',


In [17]:
def parse_features(x):
    output_dict = {}
    
    if str(x) == 'nan':
        return output_dict

    features = literal_eval(x.replace('\\"','"'))

    for item in features:
        key = item['key'].lower().strip()
        value = item['value'][0].lower().strip()
        output_dict[key] = value

    return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [21]:
keys = set()

df['features_parsed'].map( lambda x: keys.update( x.keys() ) )

len(keys)

476

In [23]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [33]:
def get_name_feat(key):
    return 'feat_' + key

for key in tqdm_notebook(keys):
    df[get_name_feat(key)] = df.features_parsed.map( lambda feats: feats[key] if key in feats else np.nan)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [42]:
keys_stat = {}
for key in keys:
    keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [44]:
{k:v for k,v in keys_stat.items() if v > 30 }

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [93]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]


df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
    df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]
    

In [57]:
df.brand = df['brand'].map(lambda x: str(x).lower())

df[ df.brand == df.feat_brand ].shape

(8846, 531)

In [98]:
feats_cat = [x for x in df.columns if '_cat' in x and x != 'feat_catalog']
feats_cat

['brand_cat',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_charge time (minutes)_cat',
 'feat_bridge width_cat',
 'feat_smart watch_cat',
 'feat_country//region of manufacture_cat',
 'feat_item weight_cat',
 'feat_designed for_cat',
 'feat_type 2_cat',
 'feat_heart rate monitor_cat',
 'feat_shoe fastening type_cat',
 'feat_country////region of manufacture_cat',
 'feat_size/dimensions_cat',
 'feat_construction_cat',
 'feat_bridge_cat',
 'feat_global composite sports type_cat',
 'feat_product name_cat',
 'feat_jacket length_cat',
 'feat_location - country_cat',
 'feat_stylenumber_cat',
 'feat_fits most screen size_cat',
 'feat_finish_cat',
 'feat_lined_cat',
 'feat_face shape_cat',
 'feat_has adaptive lenses_cat',
 'feat_wash_cat',
 'feat_stone type_cat',
 'feat_flame resistant_cat',
 'feat_navy blue_cat',
 'feat_is energy star-certified_cat',
 'feat_walmart no._cat',
 'f

In [99]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']
feats += feats_cat
feats = list( set(feats) )

model = RandomForestRegressor(max_depth = 5, n_estimators = 100)
run_model(feats, model)

(-57.603937015140374, 4.242179973279301)

In [100]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth = 5, n_estimators = 100, random_state = 0)
m.fit(X, y)

perm = PermutationImportance(m, random_state = 1).fit(X, y)
eli5.show_weights(perm, feature_names = feats)

Weight,Feature
0.2419 ± 0.0081,brand_cat
0.1002 ± 0.0093,feat_material_cat
0.0118 ± 0.0014,feat_weight_cat
0.0114 ± 0.0007,feat_brand_cat
0.0096 ± 0.0008,feat_fabric content_cat
0.0075 ± 0.0028,feat_adjustable_cat
0.0071 ± 0.0016,feat_resizable_cat
0.0057 ± 0.0004,feat_shoe category_cat
0.0048 ± 0.0009,feat_color_cat
0.0044 ± 0.0003,feat_fabric material_cat


In [116]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 
         'feat_weight_cat', 'feat_style_cat']
model = RandomForestRegressor(max_depth = 5, n_estimators = 100)
result = run_model(feats, model)

In [118]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth = 5, n_estimators = 100, random_state = 0)
m.fit(X, y)
print (result)
perm = PermutationImportance(m, random_state = 1).fit(X, y)
eli5.show_weights(perm, feature_names = feats)

(-56.94278792469422, 4.313534057436494)


Weight,Feature
0.2563 ± 0.0098,brand_cat
0.0989 ± 0.0072,feat_material_cat
0.0458 ± 0.0047,feat_gender_cat
0.0200 ± 0.0011,feat_brand_cat
0.0137 ± 0.0010,feat_weight_cat
0.0064 ± 0.0011,feat_style_cat


In [91]:
df[ df['brand'] == 'nike' ].features_parsed.sample(5).values

array([{'style': 'basketball shoes', 'country/region of manufacture': 'unknown', 'condition': 'new with box'},
       {'style': 'athletic sneakers'},
       {'sport': 'running shoes', 'occasion': 'running shoes', 'material': 'mesh', 'gender': 'men', 'shoe size': '8.5 us men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'mesh', 'color': 'midnight turq/clr jd blk white', 'model': '807144 301', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'running shoes', 'manufacturer part number': '807144 301', 'brand': 'nike', 'shoe width': 'd(m)'},
       {'main color': 'red', 'type': 'cleats'},
       {'sport': 'soccer', 'main color': 'orange'}], dtype=object)

In [92]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
toddler                4
infant                 4
mens                   4
boys'                  3
men||women             2
women                  2
youth                  2
women ,�� unisex       2
12 up                  1
adult ,�� teen         1
Name: feat_age group, dtype: int64

In [119]:
!git add matrix_one/dat

data/            GithubUpdate.ipynb  LICENSE      README.md
day4_meta.ipynb  HelloGithub.ipynb   matrix_one/
